In [5]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# for the model
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, GRU, RepeatVector, TimeDistributed, Reshape
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

Using TensorFlow backend.


In [23]:
# load files from pre-processing

X = np.load('X_cleaned.npy') #rainpredictions, hour of day, month of year, holiday, ones
y = np.load('Y_cleaned.npy')
#TimeHour = pd.read_csv('TimeHour.csv')

In [24]:
# split dataset in test and train

def ReshapeXData(dataframe):
    n_hours = 24
    n_features = dataframe.shape[1]
    dataframe_shaped = np.reshape(dataframe,(-1, n_hours, n_features))
    return dataframe_shaped

def ReshapeYData(dataframe):
    n_hours = 24
    n_features = 1
    dataframe_shaped = np.reshape(dataframe,(-1, n_hours, n_features))
    return dataframe_shaped

X_train_gru = X[0:10800,:]
X_test_gru = X[10801:13609,:]
Y_train_gru = y[0:10800]
Y_test_gru = y[10801:13609]

X_train_gru_shaped=ReshapeXData(X_train_gru)
X_test_gru_shaped = ReshapeXData(X_test_gru)
Y_train_gru_shaped = ReshapeYData(Y_train_gru)
Y_test_gru_shaped = ReshapeYData(Y_test_gru)
X_shaped = ReshapeXData(X[0:13608,:])
Y_shaped = ReshapeYData(y[0:13608])

In [ ]:
# basic model

from keras.layers import Dense

lr=0.0015
epochs=100
batch_size=32
validation_split=0.1

print('Build model...')

n_timesteps, n_features, n_outputs = X_train_gru_shaped.shape[1], X_train_gru_shaped.shape[2], Y_train_gru_shaped.shape[1]
#n_timesteps, n_features, n_outputs = X_train_gru.shape[0], X_train_gru.shape[1], Y_train_gru.shape[0]
# reshape output into [samples, timesteps, features]

model = Sequential()
#model.add(Reshape((1, X_train_gru.shape[1])))
model.add(GRU(200, activation='relu', input_shape=(n_timesteps, n_features)))
#model.add(GRU(200, activation='tanh', input_shape=(1,X_train_gru.shape[1])))
model.add(RepeatVector(n_outputs))
model.add(GRU(200, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(100, activation='relu')))
model.add(TimeDistributed(Dense(1)))
opt=optimizers.Adam(learning_rate=lr)
model.compile(loss='mae', optimizer=opt)

model.fit(
        X_train_gru_shaped,
        Y_train_gru_shaped,
        epochs=epochs,
        batch_size=batch_size,
        verbose=2,
        validation_split=0.2,
        callbacks=[
            ReduceLROnPlateau(
                verbose=1)])
    
    
#model = Sequential()
#model.add(Dense(1, input_dim=X.shape[1]))
#model.compile(optimizer = optimizers.Adam(lr=lr), loss='mean_squared_error')
#model.fit(X_train, y_train,
#          validation_split=validation_split,
#          epochs=epochs,
#          batch_size=batch_size,
#          shuffle=False)

Build model...
Train on 360 samples, validate on 90 samples
Epoch 1/100
 - 1s - loss: 130.3834 - val_loss: 132.0858
Epoch 2/100
 - 1s - loss: 122.8319 - val_loss: 138.3440
Epoch 3/100
 - 1s - loss: 123.2222 - val_loss: 133.1267
Epoch 4/100
 - 1s - loss: 102.4894 - val_loss: 100.9004
Epoch 5/100
 - 1s - loss: 85.5379 - val_loss: 91.4728
Epoch 6/100
 - 1s - loss: 79.1615 - val_loss: 81.8278
Epoch 7/100
 - 1s - loss: 78.1724 - val_loss: 79.4795
Epoch 8/100
 - 1s - loss: 77.5227 - val_loss: 80.4777
Epoch 9/100
 - 1s - loss: 75.8785 - val_loss: 78.4444
Epoch 10/100
 - 1s - loss: 80.7911 - val_loss: 81.6644
Epoch 11/100
 - 1s - loss: 76.1703 - val_loss: 80.2055
Epoch 12/100
 - 1s - loss: 75.2672 - val_loss: 78.5651
Epoch 13/100
 - 1s - loss: 75.0273 - val_loss: 81.0017
Epoch 14/100
 - 1s - loss: 73.3229 - val_loss: 78.2599
Epoch 15/100
 - 1s - loss: 71.7678 - val_loss: 76.9885
Epoch 16/100
 - 1s - loss: 69.4378 - val_loss: 78.5187
Epoch 17/100
 - 1s - loss: 67.9889 - val_loss: 78.8763
Epoch 

In [111]:
# make a prediction to test how well the model preforms
y_predicted = model.predict(X_test_gru_shaped)

In [ ]:
### LZS: WE HAVE TO IMPLEMENT HERE SOME METRICS AND A LOT OF PLOTS SO THAT WE CAN SEE HOW WELL THE MODEL PERFORMS
### LZS: AND ALSO TO BE ABLE TO FIND NEW FEATURES TO IMPROVE THE MODEL
### LZS: AND TO EVALUATE IF THE MODEL INDEED IMPROVES WHEN ADDING MORE FEATURES/ OTHER LAYERS

In [112]:
# show the inputs, predicted outputs and real outputs
for i in range(len(X_test_gru_shaped[0])):
        for j in range(len(X_test_gru_shaped[1])):
            print("Predicted=%s, Original=%s" % (y_predicted[i][j], Y_test_gru_shaped[i][j]))

Predicted=[81.856514], Original=[87.90307335]
Predicted=[105.07606], Original=[119.33287184]
Predicted=[98.98018], Original=[70.18270864]
Predicted=[93.86582], Original=[82.55571344]
Predicted=[91.243], Original=[104.59264897]
Predicted=[90.05188], Original=[72.29804405]
Predicted=[89.52695], Original=[81.48252744]
Predicted=[89.33276], Original=[0.02796992]
Predicted=[89.302155], Original=[0.1253824]
Predicted=[89.35902], Original=[0.3472128]
Predicted=[89.461426], Original=[79.76302907]
Predicted=[89.586655], Original=[80.90010568]
Predicted=[89.72199], Original=[0.08824992]
Predicted=[89.86051], Original=[85.22198364]
Predicted=[89.99861], Original=[139.90204528]
Predicted=[90.134514], Original=[94.68862363]
Predicted=[90.26776], Original=[117.25686593]
Predicted=[90.398575], Original=[90.74470117]
Predicted=[90.52677], Original=[90.482204]
Predicted=[90.65199], Original=[87.95206821]
Predicted=[90.77402], Original=[0.09789472]
Predicted=[90.89278], Original=[83.73856284]
Predicted=

In [117]:
# Create a dataframe with a prediction for all datapoints for the csv
y_predicted_all = model.predict(X_shaped)
y_pred_all = np.reshape(y_predicted_all, (13608, 1))
print(y_pred_all.shape)
y_pred_all = pd.DataFrame(data=y_pred_all)
print(TimeHour.shape)
my_df = pd.DataFrame({'Time': TimeHour.loc[:,'TimeHour'].tolist(), 'pred_y': y_pred_all.loc[:,0].tolist()})
my_df.to_csv('FirstPredictions.csv')

(13608, 1)
(13608, 2)


In [65]:
df_explore = pd.DataFrame(data=X)
df_explore['TimeHour'] = TimeHour.loc[:, 'TimeHour']

In [ ]:
#date = df_explore.loc[2, "TimeHour"]
#df_explore['TimeHour2'] = pd.to_datetime(df_explore['TimeHour'], format='%Y-%m-%d %H:%M:%S')
#date = df_explore.loc[2, "TimeHour2"]
#type(date)

In [ ]:
#new_date = date + timedelta(hours=1)
#new_date

In [66]:
df_test = pd.DataFrame(data=X)
df_test['TimeHour'] = TimeHour.loc[:, 'TimeHour']

small_df_test = df_explore[545:553]
small_df_test

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,TimeHour
545,0.017310,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-24 05:00:00
546,0.007029,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-24 06:00:00
547,0.007944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-24 07:00:00
548,0.002714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-24 08:00:00
549,0.006858,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-24 09:00:00
550,0.136719,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-24 10:00:00
551,0.371582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-25 14:00:00
552,0.091797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-25 15:00:00


In [67]:
small_df_test['TimeHour2'] = 'NA'
small_df_test['TimeHour2'] = pd.to_datetime(df_test['TimeHour'], format='%Y-%m-%d %H:%M:%S')
small_df_test['Diff'] = 'NA'
small_df_test['Before']="NA"

C:\ProgramData\Anaconda3\envs\Keras\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\envs\Keras\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\envs\Keras\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

In [68]:
# noinspection JupyterKernel
for x in range(546, 553):
    before = small_df_test.loc[x - 1, 'TimeHour2']
    small_df_test.loc[x, 'Before'] = before
    now = small_df_test.loc[x, 'TimeHour2']
    diff = now - before
    small_df_test.loc[x, 'Diff'] = diff
small_df_test

C:\ProgramData\Anaconda3\envs\Keras\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,TimeHour,TimeHour2,Diff,Before
545,0.017310,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-24 05:00:00,2018-01-24 05:00:00,NA,NA
546,0.007029,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-24 06:00:00,2018-01-24 06:00:00,0 days 01:00:00,2018-01-24 05:00:00
547,0.007944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-24 07:00:00,2018-01-24 07:00:00,0 days 01:00:00,2018-01-24 06:00:00
548,0.002714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-24 08:00:00,2018-01-24 08:00:00,0 days 01:00:00,2018-01-24 07:00:00
549,0.006858,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-24 09:00:00,2018-01-24 09:00:00,0 days 01:00:00,2018-01-24 08:00:00
550,0.136719,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-24 10:00:00,2018-01-24 10:00:00,0 days 01:00:00,2018-01-24 09:00:00
551,0.371582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-25 14:00:00,2018-01-25 14:00:00,1 days 04:00:00,2018-01-24 10:00:00
552,0.091797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2018-01-25 15:00:00,2018-01-25 15:00:00,0 days 01:00:00,2018-01-25 14:00:00


In [69]:
small_df_test = small_df_test.drop([545])
small_df_test['diff_hours'] = [x.total_seconds() / 3600 for x in small_df_test['Diff']]

In [70]:
small_df_test = small_df_test.reset_index()
small_df_test = small_df_test.drop(columns=["index"])
small_df_test

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,TimeHour,TimeHour2,Diff,Before,diff_hours
0,0.007029,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 06:00:00,2018-01-24 06:00:00,0 days 01:00:00,2018-01-24 05:00:00,1.0
1,0.007944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 07:00:00,2018-01-24 07:00:00,0 days 01:00:00,2018-01-24 06:00:00,1.0
2,0.002714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 08:00:00,2018-01-24 08:00:00,0 days 01:00:00,2018-01-24 07:00:00,1.0
3,0.006858,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 09:00:00,2018-01-24 09:00:00,0 days 01:00:00,2018-01-24 08:00:00,1.0
4,0.136719,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-24 10:00:00,2018-01-24 10:00:00,0 days 01:00:00,2018-01-24 09:00:00,1.0
5,0.371582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-25 14:00:00,2018-01-25 14:00:00,1 days 04:00:00,2018-01-24 10:00:00,28.0
6,0.091797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-25 15:00:00,2018-01-25 15:00:00,0 days 01:00:00,2018-01-25 14:00:00,1.0


In [71]:
type(small_df_test.loc[1, "diff_hours"])

numpy.float64

In [15]:
def fill_with_average(df, previous, following):
    value_new = list(df.loc[previous])
    value_previous = list(df.loc[previous])
    value_following = list(df.loc[following])
    value_new[0]= (value_previous[0]+value_following[0])/2
    date = df.loc[previous, 'TimeHour2']
    new_date = date + timedelta(hours=1)
    value_new[40]=new_date
    value_new[date.hour]=0.0
    value_new[new_date.hour]=1.0
    #value_new[43]=1.0
    df_length = len(df)
    df.loc[df_length] = value_new
    df = df.sort_values(by= "TimeHour2")
    df = df.reset_index()
    df = df.drop(columns=["index"])
    return df

def Y_fill_with_average(df, previous, following):
    value_new = list(df.loc[previous])
    value_previous = list(df.loc[previous])
    value_following = list(df.loc[following])
    value_new[0]= (value_previous[0]+value_following[0])/2
    date = df.loc[previous, 'TimeHour2']
    print(date)
    new_date = date + timedelta(hours=1)
    value_new[2]=new_date
    #value_new[date.hour]=0.0
    #value_new[new_date.hour]=1.0
    #value_new[43]=1.0
    df_length = len(df)
    df.loc[df_length] = value_new
    df = df.sort_values(by= "TimeHour2")
    df = df.reset_index()
    df = df.drop(columns=["index"])
    return df



In [10]:
df_test = pd.DataFrame(data=y)
df_test['TimeHour'] = TimeHour.loc[:, 'TimeHour']
df_test['TimeHour2'] = 'NA'
df_test['TimeHour2'] = pd.to_datetime(df_test['TimeHour'], format='%Y-%m-%d %H:%M:%S')
df_test['Diff'] = 'NA'
df_test['Before']="NA"
for x in range(1, df_test.shape[0]):
    before = df_test.loc[x - 1, 'TimeHour2']
    df_test.loc[x, 'Before'] = before
    now = df_test.loc[x, 'TimeHour2']
    diff = now - before
    df_test.loc[x, 'Diff'] = diff
df_test
df_test = df_test.drop(0)
df_test['diff_hours'] = [x.total_seconds() / 3600 for x in df_test['Diff']]



df_test = df_test.drop([1141,1142])
df_test = df_test.reset_index()
df_test = df_test.drop(columns=["index"])
df_test = df_test.drop([655,656,657,658,659,660,661,662,663,664,665,666,667,668,669])
df_test = df_test.reset_index()
df_test = df_test.drop(columns=["index"])
df_test = df_test.drop([641,642,643,644,645,646,647,648,649,650,651,652,653,654,655])
df_test = df_test.reset_index()
df_test = df_test.drop(columns=["index"])
df_test = df_test.drop([1480])
df_test = df_test.drop([7900,7901,7902,7903,7904,7905])
df_test = df_test.reset_index()
df_test = df_test.drop(columns=["index"])
df_test[df_test['diff_hours']>1]


,0,TimeHour,TimeHour2,Diff,Before,diff_hours
550,47.628271,2018-01-25 14:00:00,2018-01-25 14:00:00,1 days 04:00:00,2018-01-24 10:00:00,28.0
662,0.000000,2018-02-05 16:00:00,2018-02-05 16:00:00,4 days 05:00:00,2018-02-01 11:00:00,101.0
1037,76.974656,2018-02-23 09:00:00,2018-02-23 09:00:00,2 days 03:00:00,2018-02-21 06:00:00,51.0
1110,91.311921,2018-02-28 13:00:00,2018-02-28 13:00:00,0 days 04:00:00,2018-02-28 09:00:00,4.0
1479,35.789604,2018-04-05 20:00:00,2018-04-05 20:00:00,20 days 23:00:00,2018-03-15 21:00:00,503.0
1661,0.000000,2018-04-13 16:00:00,2018-04-13 16:00:00,0 days 06:00:00,2018-04-13 10:00:00,6.0
1774,45.289669,2018-04-18 17:00:00,2018-04-18 17:00:00,0 days 09:00:00,2018-04-18 08:00:00,9.0
1793,45.334073,2018-04-20 12:00:00,2018-04-20 12:00:00,1 days 01:00:00,2018-04-19 11:00:00,25.0
2133,46.871539,2018-05-04 18:00:00,2018-05-04 18:00:00,0 days 03:00:00,2018-05-04 15:00:00,3.0
3747,43.357102,2018-07-11 01:00:00,2018-07-11 01:00:00,0 days 02:00:00,2018-07-10 23:00:00,2.0


In [16]:
x = 0
while x < 13000:
    if df_test.loc[x, 'diff_hours'] != 1.0 and df_test.loc[x, "diff_hours"] != 24.0 and df_test.loc[x, 'diff_hours'] != 96.0 and df_test.loc[x, 'diff_hours'] != 48.0 and df_test.loc[x, 'diff_hours'] != 480.0:
        print(x)
        df_test.loc[x, 'diff_hours'] = df_test.loc[x, 'diff_hours'] - 1
        df_test = Y_fill_with_average(df_test, x-1, x)
        x = x - 1
    else:
        x = x + 1

print ('Missing values approximated')
df_test[df_test['diff_hours']>1]

550
2018-01-24 10:00:00
551
2018-01-24 11:00:00
664
2018-02-01 11:00:00
665
2018-02-01 12:00:00
666
2018-02-01 13:00:00
667
2018-02-01 14:00:00
668
2018-02-01 15:00:00
1044
2018-02-21 06:00:00
1045
2018-02-21 07:00:00
1046
2018-02-21 08:00:00
1120
2018-02-26 09:00:00
1121
2018-02-26 10:00:00
1122
2018-02-26 11:00:00
1492
2018-03-15 21:00:00
1493
2018-03-15 22:00:00
1494
2018-03-15 23:00:00
1495
2018-03-16 00:00:00
1496
2018-03-16 01:00:00
1497
2018-03-16 02:00:00
1498
2018-03-16 03:00:00
1499
2018-03-16 04:00:00
1500
2018-03-16 05:00:00
1501
2018-03-16 06:00:00
1502
2018-03-16 07:00:00
1503
2018-03-16 08:00:00
1504
2018-03-16 09:00:00
1505
2018-03-16 10:00:00
1506
2018-03-16 11:00:00
1507
2018-03-16 12:00:00
1508
2018-03-16 13:00:00
1509
2018-03-16 14:00:00
1510
2018-03-16 15:00:00
1511
2018-03-16 16:00:00
1512
2018-03-16 17:00:00
1513
2018-03-16 18:00:00
1514
2018-03-16 19:00:00
1697
2018-04-13 10:00:00
1698
2018-04-13 11:00:00
1699
2018-04-13 12:00:00
1700
2018-04-13 13:00:00
1701
20

,0,TimeHour,TimeHour2,Diff,Before,diff_hours
552,47.628271,2018-01-25 14:00:00,2018-01-25 14:00:00,1 days 04:00:00,2018-01-24 10:00:00,24.0
669,0.000000,2018-02-05 16:00:00,2018-02-05 16:00:00,4 days 05:00:00,2018-02-01 11:00:00,96.0
1047,76.974656,2018-02-23 09:00:00,2018-02-23 09:00:00,2 days 03:00:00,2018-02-21 06:00:00,48.0
1515,35.789604,2018-04-05 20:00:00,2018-04-05 20:00:00,20 days 23:00:00,2018-03-15 21:00:00,480.0
1843,45.334073,2018-04-20 12:00:00,2018-04-20 12:00:00,1 days 01:00:00,2018-04-19 11:00:00,24.0


In [20]:

#df_test = df_test.drop(['Diff', 'TimeHour', 'TimeHour2', 'diff_hours', 'Before'],axis=1)
#df_test = df_test.drop([13632,13633,13634,13635,13636,13637,13638,13639,13640])
df_test = df_test.drop([13632,13633,13634,13635,13636,13637,13638])

print(df_test.shape)
df_test

(13632, 1)


,0
0,502.995390
1,501.561763
2,267.722037
3,164.684623
4,415.606068
...,...
13627,107.449515
13628,184.320927
13629,84.346758
13630,82.764828


In [22]:
np.save('Y_cleaned.npy', df_test)